## Task 0
Data ingestion (sqlite to duckdb)

In [ ]:
# import liabraries
# duckdb helps to create a duckdb database
# pandas use to load and move the data
# sqlite to read database

import sqlite3
import duckdb
import pandas  as pd
import matplotlib.pyplot as plt 

# Connect to SQLite Sakila database
sqlite_conn = sqlite3.connect("sakila.db")  # make sure path is correct

### Connect to sqlite sakila database

In [6]:
# get real tables from sqlite (ignore sqlite_master)
tables = pd.read_sql("""
SELECT name 
FROM sqlite_master 
WHERE type='table' 
AND name NOT LIKE 'sqlite_%'
""", sqlite_conn)

In [7]:
for table_name in tables['name']:
    df = pd.read_sql(f"SELECT * FROM {table_name}", sqlite_conn)
    duckdb_conn.execute(f"CREATE OR REPLACE TABLE {table_name} AS SELECT * FROM df")


### Connect to duckdb database

In [9]:
# Connect to duckdb (creates file if it does not exist)
duckdb_conn = duckdb.connect("sakila.duckdb")

print("Connected to duckdb database")

Connected to duckdb database


In [10]:
duckdb_conn.execute("SHOW TABLES").df()

,name


In [ ]:
# .................
# # Connect to sqlite database (replace with ur file path)
# sqlite_conn = sqlite3.connect("sakila.db")

# print("Connected to sqlite database")

# # Check the tables in sqlite
# tables = pd.read_sql(
#     "SELECT name FROM sqlite_master WHERE type='table';", 
#     sqlite_conn
# )

# tables 

Connected to sqlite database


,name


<!-- ### Connect to duckdb database -->

In [ ]:
# # Connect to duckdb (creates file if it does not exist)
# duckdb_conn = duckdb.connect("sakila.duckdb")

# print("Connected to duckdb database")

Connected to duckdb database


### Copy tables from sqlite to duckdb

In [11]:
# Now we will read each table from sqlite, and store it as a duckdb table

for table_name in tables['name']:
    # read table from sqlite into pandas
    df = pd.read_sql(f"SELECT * FROM {table_name}", sqlite_conn)

    # creating a table in duckdb
    duckdb_conn.execute(f"CREATE TABLE {table_name} AS SELECT * FROM df")

    print(f"copied table: {table_name}")

### Verify tables in duckdb (lets confirm)

In [12]:
# Show tables in duckdb 
duckdb_tables = duckdb_conn.execute("SHOW TABLES").df()

duckdb_tables 

,name


### Test a query from duckdb (first 5 films)

In [13]:
duckdb_conn.execute("SELECT * FROM sqlite_master LIMIT 5").df()

,type,name,tbl_name,rootpage,sql


### Close database connections when done

In [14]:
sqlite_conn.close()
duckdb_conn.close()

print("Connections closed")

Connections closed


## Task  1 -     Exploratory data analysis (EDA)

#### a) Which movies are longer than 3hours (180 min)?
- Show:  title & length

In [15]:
import duckdb

db = duckdb.connect("sakila.duckdb")

In [20]:
# Select movies longer than 180 minutes
query = """ 
SELECT title, length
FROM film           
WHERE length > 180
ORDER BY length DESC 
"""

movies_longer_than_3h = db.execute(query).df()
movies_longer_than_3h 

CatalogException: Catalog Error: Table with name film does not exist!
Did you mean "sqlite_master"?

LINE 3: FROM film           
             ^

#### b) Movies with the word "love" in the title

In [ ]:
# Find movies that contain 'love' in the title
query = """ 
SELECT title, rating, length, description
FROM film
WHERE LOWER(title) LIKE '%love%'
"""

love_movies = con.execute(query).df()
love_movies

#### c) Movies length statistics
We calculate:
- shortest
- average
- median
- longest

In [ ]:
query = """ 
SELECT
    MIN(length) AS shortest,
    AVG(length) AS average,
    MEDIAN(length) AS median,
    MAX(length) AS longest
FROM film
"""

movie_stats = con.execute(query).df()
movie_stats

#### d) 10 most expensive movies to rent per day

In [ ]:
cost per day = rental_rate / rental_duration

query = """ 
SELECT title,
       rental_rate,
       rental_duration,
       rental_rate / rental_duration AS cost_per_day
FROM film
ORDER BY cost_per_day DESC
LIMIT 10
"""

expensive_movies = con.execute(query).df()
expensive_movies

#### e) Top 10 actors with the number of movies they have played in

In [ ]:
query = """ 
SELECT
    a.first_name || '' || a.last_name AS actor_name,
    COUNT(fa.film_id) AS movie_count
FROM actor a
JOIN film_actor fa ON a.actor_id = fa.actor_id
GROUP BY actor_name
ORDER BY movie_count DESC
LIMIT 10
"""

top_actors = con.execute(query).df()
top_actors 

#### f) My own 3 simple questions


Q_1 : Which categories have most movies?

In [6]:
query = """ 
SELECT c.name AS category, COUNT(*) AS movies
FROM category c
JOIN film_category fc ON c.category_id = fc.category_id
GROUP BY category
ORDER BY movies DESC
"""

category_movies = con.execute(query).df()
category_movies 

CatalogException: Catalog Error: Table with name category does not exist!
Did you mean "pg_attrdef"?

LINE 3: FROM category c
             ^

Q_2: Average rental rate per category

In [ ]:
query = """ 
SELECT c.name AS category, AVG(f.rental_rate) AS avg_rate
FROM category c
JOIN film_category fc ON c.category_id = fc.category_id
JOIN film f ON fc.film_id = f.film_id
GROUP BY category
"""

avg_rates = con.execute(query).df()
avg_rates

Q_3: Top 5 customers by number of rentals

In [ ]:
query = """ 
SELECT c.first_name || '' || c.last_name AS customer,
       COUNT(r.rental_id) AS rentals
FROM customer c
JOIN rental r ON c.customer_id = r.customer_id
GROUP BY customer
ORDER BY rentals DESC
LIMIT 5
"""

top_renters = con.execute(query).df()
top_renters

## Task 2 - Graphs

#### a) Top 5 customers by total spend

In [7]:
query = """ 
SELECT c.first_name || '' || c.last_name AS customer,
       SUM(p.amount) AS total_spend
FROM customer c
JOIN payment p ON c.customer_id = p.customer_id
GROUP BY customer
ORDER BY total_spend DESC
LIMIT 5
"""

top_spend = con.execute(query).df()

# Bar chart
plt.figure()
plt.bar(top_spend['customer'], top_spend['total_spend'])
plt.title('Top 5 Customers by Total Spend')
plt.xlabel('Customer')
plt.ylabel('Total Spend')
plt.show()

CatalogException: Catalog Error: Table with name customer does not exist!
Did you mean "pg_constraint or sqlite_master"?

LINE 4: FROM customer c
             ^

#### b) Revenue per film category

In [8]:
query = """ 
SELECT c.name AS category,
       SUM(p.amount) AS revenue
FROM category c
JOIN film_category fc ON c.category_id = fc.category_id
JOIN inventory i ON fc.film_id = i.film_id
JOIN rental r ON i.inventory_id = r.inventory_id
JOIN payment p ON r.rental_id = p.rental_id
GROUP BY revenue DESC
"""

revenue_category = con.execute(query).df()

plt.figure()
plt.bar(revenue_category['category'], revenue_category['revenue'])
plt.title('Revenue per Film Category')
plt.xlabel('Category')
plt.ylabel('Revenue')
plt.xticks(rotation=45)
plt.show()

ParserException: Parser Error: syntax error at or near "DESC"

LINE 9: GROUP BY revenue DESC
                         ^